## *Tarea M52 - Bryan Alberto Coronado García*
---

### Configuración de plataforma Spark

In [1]:
# Importación de librerías para la practica de Spark
import sys
from operator import add
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")

# Configuración de Spark
spark = SparkSession.builder.appName("M52").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/31 17:00:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Importación de datos de Housing a una estructura Spark

In [2]:
# Generación de variable para la ruta del archivo
path_archivo = '/Users/bryancoronado/Desktop/Housing.csv'

# Generacion del DataFrame a partir del archivo CSV (con headers para facilitar la lectura)
df = spark.read.csv(path_archivo, header=True)

# Verificación de la carga del DataFrame
df.show(10)

+--------+-----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|   price| area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+--------+-----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|13300000| 7420|       4|        2|      3|     yes|       no|      no|             no|            yes|      2|     yes|       furnished|
|12250000| 8960|       4|        4|      4|     yes|       no|      no|             no|            yes|      3|      no|       furnished|
|12250000| 9960|       3|        2|      2|     yes|       no|     yes|             no|             no|      2|     yes|  semi-furnished|
|12215000| 7500|       4|        2|      2|     yes|       no|     yes|             no|            yes|      3|     yes|       furnished|
|11410000| 7420|       4|        1

In [3]:
# Verificacion del tipo de datos
print("Tipo de datos del DataFrame:")
type(df)

Tipo de datos del DataFrame:


pyspark.sql.classic.dataframe.DataFrame

In [4]:
# Transformación de los tipos de datos para asegurar que son correctos

from pyspark.sql.types import IntegerType, FloatType

df = df.withColumn("price", df.price.cast(IntegerType()))
df = df.withColumn("area", df.area.cast(IntegerType()))
df = df.withColumn("bedrooms", df.bedrooms.cast(IntegerType()))
df = df.withColumn("bathrooms", df.bathrooms.cast(IntegerType()))
df = df.withColumn("stories", df.stories.cast(IntegerType()))
df = df.withColumn("parking", df.parking.cast(IntegerType()))

df.printSchema()

root
 |-- price: integer (nullable = true)
 |-- area: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- stories: integer (nullable = true)
 |-- mainroad: string (nullable = true)
 |-- guestroom: string (nullable = true)
 |-- basement: string (nullable = true)
 |-- hotwaterheating: string (nullable = true)
 |-- airconditioning: string (nullable = true)
 |-- parking: integer (nullable = true)
 |-- prefarea: string (nullable = true)
 |-- furnishingstatus: string (nullable = true)



### Selección de datos de housing con filtros simples

#### 1) Listado completo de columnas ordenado por zipcode

In [ ]:
import pyspark.sql.functions as F1

# Se toma la columna "area" como sustituto para "zipcode" ya que no se encuentra en el dataset
df.sort(F1.col("area")).show()

+-------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|  price|area|bedrooms|bathrooms|stories|mainroad|guestroom|basement|hotwaterheating|airconditioning|parking|prefarea|furnishingstatus|
+-------+----+--------+---------+-------+--------+---------+--------+---------------+---------------+-------+--------+----------------+
|3150000|1650|       3|        1|      2|      no|       no|     yes|             no|             no|      0|      no|     unfurnished|
|1890000|1700|       3|        1|      2|     yes|       no|      no|             no|             no|      0|      no|     unfurnished|
|2275000|1836|       2|        1|      1|      no|       no|     yes|             no|             no|      0|      no|  semi-furnished|
|4340000|1905|       5|        1|      2|      no|       no|     yes|             no|             no|      0|      no|  semi-furnished|
|3430000|1950|       3|        2|      2|     ye

#### 2) Listado para el zipcode con mayor número de casas, calcular el promedio de precio, y tamaño en m2

In [ ]:
df.groupBy("area").agg( \
    F1.count("*").alias("num_casas"),
    F1.avg("price").alias("precio_promedio")) \
    .orderBy(F1.desc("num_casas")).show()

# NOTA: No es posible realizar una columna del tamano de la casa, ya que no existe la data correspondiente.
# El punto actualizado es "Listado de zipcodes ordenados por el número de casas, y el promedio de precio"

+----+---------+------------------+
|area|num_casas|   precio_promedio|
+----+---------+------------------+
|6000|       24| 7051479.166666667|
|3000|       14|         3309000.0|
|4500|       13|4031192.3076923075|
|4000|       11| 4040272.727272727|
|5500|        9| 5762555.555555556|
|6600|        9| 6443111.111111111|
|3600|        8|         3360437.5|
|3180|        7|         3530000.0|
|4040|        7|         4139000.0|
|6360|        7|         5604000.0|
|3640|        7|         3542000.0|
|3630|        7|         3515000.0|
|3500|        6| 4275833.333333333|
|5400|        6| 4631666.666666667|
|2145|        6|3606166.6666666665|
|3450|        5|         3680600.0|
|3850|        5|         3136000.0|
|3480|        5|         3227000.0|
|4800|        5|         5742800.0|
|3520|        5|         4107600.0|
+----+---------+------------------+
only showing top 20 rows


#### 3) Agrupamiento en Spark, por número de habitaciones y baños, del precio.  Ej: # habitaciones | # baños | precio promedio, esto por zipcode.

In [18]:
df.groupBy("area").agg( \
    F1.count("bedrooms").alias("num_habitaciones"),
    F1.count("bathrooms").alias("num_baños"),
    F1.avg("price").alias("precio_promedio")) \
    .orderBy(F1.desc("num_habitaciones"), F1.desc("num_baños")).show(20)

+----+----------------+---------+------------------+
|area|num_habitaciones|num_baños|   precio_promedio|
+----+----------------+---------+------------------+
|6000|              24|       24| 7051479.166666667|
|3000|              14|       14|         3309000.0|
|4500|              13|       13|4031192.3076923075|
|4000|              11|       11| 4040272.727272727|
|5500|               9|        9| 5762555.555555556|
|6600|               9|        9| 6443111.111111111|
|3600|               8|        8|         3360437.5|
|3180|               7|        7|         3530000.0|
|4040|               7|        7|         4139000.0|
|6360|               7|        7|         5604000.0|
|3640|               7|        7|         3542000.0|
|3630|               7|        7|         3515000.0|
|3500|               6|        6| 4275833.333333333|
|5400|               6|        6| 4631666.666666667|
|2145|               6|        6|3606166.6666666665|
|3450|               5|        5|         3680